In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
wdir = '../data'
outdir = wdir + '/processed'

In [3]:
def dump_excel_to_df(excel_file, sheet_name, skip_rows, last_row, respitory_name, n_weeks, delete_row1=False, custom_keepcols=None):
    #Read the Excel file and traspose it
    df = pd.read_excel(wdir + '/{}'.format(excel_file),
                   sheet_name=sheet_name, 
                   skiprows=skip_rows).T
    #Keep data up to last row last_row specified
    df = df.iloc[:,0:last_row].copy(deep=True)
    #Get the columns names, the first row (element 0 in Python)
    cols = list(df.iloc[0,:])
    #Delete the first row element from df (in Python the second element of a list is 1)
    df = df.iloc[1:,:].copy(deep=True)
    #Name the column with the cols variable
    df.columns = cols
    
    #If there is twice the name of the variable (it is in some years, e.g. 2019), delete it
    if delete_row1:
        df = df.iloc[1:,:].reset_index(drop=True)
    
    #numpy.arange([start, ]stop, [step, ]dtype=None)Return evenly spaced values within a given interval.
    df['week'] = np.arange(1,n_weeks+1)
    
    if custom_keepcols:
        keepcols = custom_keepcols 
    else:
        keepcols = ['week',
                    'Week ended',
                    'Total deaths, all ages',
                    'Total deaths: average of corresponding',
                    respitory_name,
                    'North East',
                    'North West',
                    'Yorkshire and The Humber',
                    'East Midlands',
                    'West Midlands',
                    'East',
                    'London',
                    'South East',
                    'South West',
                    'Wales']   
    df = df[keepcols].reset_index(drop=True)
    
    return df

In [4]:
xlslist = ['publishedweek2010.xls',
           'publishedweek2011.xls',
           'publishedweek2012.xls',
           'publishedweek2013.xls', 
           'publishedweek2014.xls', 
           'publishedweek2015.xls', 
           'publishedweek522016.xls', 
           'publishedweek522017.xls', 
           'publishedweek522018withupdatedrespiratoryrow.xls', 
           'publishedweek522019.xls', 
           'publishedweek122020.xlsx', 
           'publishedweek122020.xlsx',
           'publishedweek1320201.xlsx',
           'publishedweek1320201.xlsx']

sheet_list = ['Weekly Figures 2010',
              'Weekly Figures 2011',
              'Weekly Figures 2012',
              'Weekly Figures 2013', 
              'Weekly Figures 2014', 
              'Weekly Figures 2015', 
              'Weekly figures 2016', 
              'Weekly figures 2017', 
              'Weekly figures 2018', 
              'Weekly figures 2019',
              'Weekly figures 2020', 
              'Covid-19 - Weekly figures 2020',
              'Weekly figures 2020',
              'Covid-19 - Weekly figures 2020']

skip_list = [3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3]

last_row_list = [48, 49, 48, 48, 49, 48, 48, 48, 48, 48, 50, 50, 50, 50]

weeks_list = [52, 52, 52, 52, 52, 53, 52, 52, 52, 52, 53, 53, 53, 53]

respitory_list = ['All respiratory diseases (ICD-10 J00-J99)  ICD-10 v 2001',
                  'ICD-10 v 2010',
                  'All respiratory diseases (ICD-10 J00-J99)  ICD-10 v 2010', 
                  'All respiratory diseases (ICD-10 J00-J99)  ICD-10 v 2010',
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)',
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)',
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)', 
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)',
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)', 
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)', 
                  'Deaths where the underlying cause was respiratory disease (ICD-10 J00-J99)', 
                  '',
                  'Deaths where the underlying cause was respiratory disease (ICD-10 J00-J99)',
                  '']

yrs = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, '2020_covid', 2020, '2020_covid']
keys = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, '2020_wk12', '2020_wk12_covid', '2020_wk13', '2020_wk13_covid']

delete_list = [False, False, False, False, False, False, True, True, True, True, True, True, True, True]


print(len(xlslist),
      len(sheet_list),
      len(skip_list),
      len(last_row_list),
      len(weeks_list),
      len(respitory_list),
      len(yrs),
      len(keys),
      len(delete_list))

14 14 14 14 14 14 14 14 14


In [5]:
idx = 12

custcols = ['week',
            'Week ended',
            'Deaths involving COVID-19, all ages1',
            'North East',
            'North West',
            'Yorkshire and The Humber',
            'East Midlands',
            'West Midlands',
            'East',
            'London',
            'South East',
            'South West',
            'Wales'] 

custcols=None

df = dump_excel_to_df(xlslist[idx],
                      sheet_list[idx],
                      skip_rows=skip_list[idx],
                      last_row=last_row_list[idx],
                      respitory_name=respitory_list[idx], 
                      n_weeks=weeks_list[idx], 
                      delete_row1=delete_list[idx],
                      custom_keepcols=custcols)
df

,week,Week ended,"Total deaths, all ages",Total deaths: average of corresponding,Deaths where the underlying cause was respiratory disease (ICD-10 J00-J99),North East,North West,Yorkshire and The Humber,East Midlands,West Midlands,East,London,South East,South West,Wales
0,1,2020-01-03 00:00:00,12254,12175,2141,673,1806,1240,1060,1349,1162,1113,1814,1225,787
1,2,2020-01-10 00:00:00,14058,13822,2477,707,1932,1339,1195,1450,1573,1272,2132,1487,939
2,3,2020-01-17 00:00:00,12990,13216,2188,647,1696,1278,1106,1407,1457,1073,2064,1466,767
3,4,2020-01-24 00:00:00,11856,12760,1893,612,1529,1187,1024,1231,1410,1028,1833,1253,723
4,5,2020-01-31 00:00:00,11612,12206,1746,561,1461,1136,1015,1262,1286,1092,1820,1233,727
5,6,2020-02-07 00:00:00,10986,11925,1572,564,1529,1072,922,1052,1259,987,1729,1157,690
6,7,2020-02-14 00:00:00,10944,11627,1602,573,1427,1059,976,1159,1172,967,1688,1169,728
7,8,2020-02-21 00:00:00,10841,11548,1619,539,1477,1087,924,1116,1167,1032,1675,1118,679
8,9,2020-02-28 00:00:00,10816,11183,1546,572,1476,1078,919,1174,1115,1085,1587,1133,651
9,10,2020-03-06 00:00:00,10895,11498,1557,568,1490,1112,930,1098,1149,982,1726,1170,652


In [6]:
df_dict = {}

In [7]:
for idx in range(14):
    
    if yrs[idx] == '2020_covid':
        
        custcols = ['week',
                    'Week ended',
                    'Deaths involving COVID-19, all ages1',
                    'North East',
                    'North West',
                    'Yorkshire and The Humber',
                    'East Midlands',
                    'West Midlands',
                    'East',
                    'London',
                    'South East',
                    'South West',
                    'Wales'] 
    else:
        
        custcols = None
    
    df = dump_excel_to_df(xlslist[idx],
                          sheet_list[idx],
                          skip_rows=skip_list[idx],
                          last_row=last_row_list[idx],
                          respitory_name=respitory_list[idx], 
                          n_weeks=weeks_list[idx], 
                          delete_row1=delete_list[idx],
                          custom_keepcols=custcols)

    df_dict[keys[idx]] = df

In [8]:
df_dict.keys()

dict_keys([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, '2020_wk12', '2020_wk12_covid', '2020_wk13', '2020_wk13_covid'])

In [10]:
for key, df in df_dict.items():
    
    if 'covid' in str(key):
    
        colnames = ['week',
                    'week_ended',
                    'covid_deaths',
                    'north_east',
                    'north_west',
                    'yorkshire_and_the_humber',
                    'east_midlands',
                    'west_midlands',
                    'east',
                    'london',
                    'south_east',
                    'south_west',
                    'wales'] 
        
    else:
        
         colnames = ['week',
                     'week_ended',
                     'total_deaths_all_ages',
                     'total_deaths_avg_corresponding_5',
                     'respitory_deaths',
                     'north_east',
                     'north_west',
                     'yorkshire_and_the_humber',
                     'east_midlands',
                     'west_midlands',
                     'east',
                     'london',
                     'south_east',
                     'south_west',
                     'wales'] 
    
    
    df.columns = colnames
    
    
    df.to_csv(outdir + '/deaths_{}.csv'.format(key), index=False)